## Rigid body 6 DOF
Devlop a system for a rigid bofy in 6 DOF and do a simualtion

In [1]:
import warnings
#warnings.filterwarnings('ignore')
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sympy as sp
import sympy.physics.mechanics as me
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from substitute_dynamic_symbols import substitute_dynamic_symbols, find_name, find_derivative_name, lambdify, find_derivatives


In [3]:
x0,y0,z0 = me.dynamicsymbols('x0 y0 z0')
x01d,y01d,z01d = me.dynamicsymbols('x0 y0 z0',1)
x02d,y02d,z02d = me.dynamicsymbols('x0 y0 z0',2)

phi,theta,psi = me.dynamicsymbols('phi theta psi')
phi1d,theta1d,psi1d = me.dynamicsymbols('phi theta psi',1)
phi2d,theta2d,psi2d = me.dynamicsymbols('phi theta psi',2)


p_x = sp.Symbol('p_x')
p_y = sp.Symbol('p_y')
p_z = sp.Symbol('p_z')



In [4]:
x02d

Derivative(x0(t), (t, 2))

In [5]:
N = me.ReferenceFrame('N')

In [6]:
S = N.orientnew('S', 'Body', [psi,theta,phi],'ZYX')

In [7]:
S.ang_vel_in(N)

(-sin(theta)*psi' + phi')*S.x + (sin(phi)*cos(theta)*psi' + cos(phi)*theta')*S.y + (-sin(phi)*theta' + cos(phi)*cos(theta)*psi')*S.z

In [8]:
S.ang_acc_in(N)

(-sin(theta)*psi'' - cos(theta)*psi'*theta' + phi'')*S.x + (-sin(phi)*sin(theta)*psi'*theta' + sin(phi)*cos(theta)*psi'' - sin(phi)*phi'*theta' + cos(phi)*cos(theta)*phi'*psi' + cos(phi)*theta'')*S.y + (-sin(phi)*cos(theta)*phi'*psi' - sin(phi)*theta'' - sin(theta)*cos(phi)*psi'*theta' + cos(phi)*cos(theta)*psi'' - cos(phi)*phi'*theta')*S.z

In [10]:
M = me.Point('M')  # Origo of measurement system
M.set_vel(N, 0)

O = me.Point('O') # Origo of ship
O.set_pos(M,x0*N.x + y0*N.y + z0*N.z)
O.set_vel(S, 0)


In [41]:
O.v1pt_theory(M,N,S)
#O.a1pt_theory(M,N,S)

(((-sin(phi)*cos(psi) + sin(psi)*sin(theta)*cos(phi))*(-sin(phi)*theta' + cos(phi)*cos(theta)*psi') + (-sin(theta)*psi' + phi')*sin(psi)*cos(theta) + (sin(phi)*sin(psi)*sin(theta) + cos(phi)*cos(psi))*(sin(phi)*cos(theta)*psi' + cos(phi)*theta'))*z0 - ((-sin(phi)*theta' + cos(phi)*cos(theta)*psi')*cos(phi)*cos(theta) - (-sin(theta)*psi' + phi')*sin(theta) + (sin(phi)*cos(theta)*psi' + cos(phi)*theta')*sin(phi)*cos(theta))*y0)*N.x + (-((sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))*(-sin(phi)*theta' + cos(phi)*cos(theta)*psi') + (-sin(theta)*psi' + phi')*cos(psi)*cos(theta) + (sin(phi)*sin(theta)*cos(psi) - sin(psi)*cos(phi))*(sin(phi)*cos(theta)*psi' + cos(phi)*theta'))*z0 + ((-sin(phi)*theta' + cos(phi)*cos(theta)*psi')*cos(phi)*cos(theta) - (-sin(theta)*psi' + phi')*sin(theta) + (sin(phi)*cos(theta)*psi' + cos(phi)*theta')*sin(phi)*cos(theta))*x0)*N.y + (((sin(phi)*sin(psi) + sin(theta)*cos(phi)*cos(psi))*(-sin(phi)*theta' + cos(phi)*cos(theta)*psi') + (-sin(theta)*psi' + phi')*c

## Mass

In [42]:
mass = sp.symbols('m')
#x_g, y_g, z_g = sp.symbols('x_g y_g z_g')

In [43]:
#c_og = me.Point('c_og')
#c_og.set_pos(O,x_g*S.x + y_g*S.y + z_g*S.z)
#c_og.set_vel(S,0)

In [44]:
#c_og.v2pt_theory(O,N,S)

## Inertia

In [45]:
I_xx, I_yy, I_zz = sp.symbols('I_xx, I_yy, I_zz')
body_inertia_dyadic = me.inertia(S, ixx=I_xx, iyy=I_yy, izz=I_zz)
body_inertia_dyadic

I_xx*(S.x|S.x) + I_yy*(S.y|S.y) + I_zz*(S.z|S.z)

In [46]:
body_inertia_dyadic.to_matrix(S)

Matrix([
[I_xx,    0,    0],
[   0, I_yy,    0],
[   0,    0, I_zz]])

In [47]:
body_central_inertia = (body_inertia_dyadic, O)

In [48]:
body = me.RigidBody('Rigid body', masscenter=O, frame = S,
                      mass=mass, inertia=body_central_inertia)

## Forces

In [49]:
fx, fy, fz, mx, my, mz = sp.symbols('f_x f_y f_z m_x m_y m_z')

In [50]:
force_vector = fx*N.x + fy*N.y + fz*N.z
torque_vector = mx*N.x + my*N.y + mz*N.z

In [51]:
force = (O, force_vector)
torque = (O, torque_vector)

## Equations of Motion

In [52]:
coordinates = [x0, y0, z0, phi, theta, psi]
speeds = [x01d, y01d, z01d, phi1d, theta1d, psi1d]

In [38]:
kinematical_differential_equations = [x01d - x0.diff(),
                                      #x02d - x01d.diff(),
                                      
                                      y01d - y0.diff(),
                                      #y02d - y01d.diff(),
                                      
                                      z01d - z0.diff(),
                                      #z02d - z01d.diff(),
                                      
                                      phi1d - phi.diff(),
                                      #phi2d - phi1d.diff(),
                                      
                                      theta1d - theta.diff(),
                                      #theta2d - theta1d.diff(),
                                      
                                      psi1d - psi.diff(),
                                      #psi2d - psi2d.diff(),                                      
                                     ]

In [39]:
?me.KanesMethod

In [40]:
kane = me.KanesMethod(N, coordinates, speeds, kinematical_differential_equations)

NotImplementedError: Underdetermined systems not supported.

In [ ]:
loads = [force,
         torque]

In [ ]:
bodies = [body]
fr, frstar = kane.kanes_equations(bodies, loads)